In [49]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import PCA
import scipy.sparse

In [96]:
df=pd.read_csv("final.csv",index_col=0)

In [97]:
df.head()

,Genre,Title,Artist,Explicit,Length,Scale,Key,BPM,Danceability,Positiveness,Liveness,Energy,speechness,KeyScaled,Lyrics,Not_Formatted_Lyrics,Lem_Lyrics,Lyrics_Length,Longest_Line,Length_Words_Ratio
0,rock,into the storm,gojira,False,5.02,D,Major,106,25,11,13,97,7,D,go say something about awakening the rebellion...,Go\r\n\r\nSay something about awakening\r\nThe...,awaken rebellion wake change course law follow...,173,47,34.462151
1,rock,the sound of silence,disturbed,False,4.08,F#,Minor,86,32,18,10,28,3,F#m,hello darkness my old friend i have come to ta...,"Hello darkness, my old friend\r\nI've come to ...",hello darkness old friend I talk vision softly...,218,97,53.431373
2,rock,amazonia,gojira,False,5.00,A,Major,182,38,51,24,95,4,A,incite a riot put yourself in a trance you rot...,"Incite a riot, put yourself in a trance\r\nYou...",incite riot put trance rotate frame world rely...,154,48,30.800000
3,rock,back in black,ac/dc,False,4.15,A,Major,188,31,76,8,70,5,A,back in black i hit the sack i have been too l...,"Back in black, I hit the sack\r\nI've been too...",back black I hit sack I long I glad back yes I...,307,55,73.975904
4,rock,another world,gojira,False,4.24,G,Minor,170,35,24,12,88,4,Gm,we mock and slaughter all the purest kinds bli...,We mock and slaughter all the purest kinds\r\n...,mock slaughter pure kind blind noise maze flas...,95,53,22.405660


In [98]:
df.columns

Index(['Genre', 'Title', 'Artist', 'Explicit', 'Length', 'Scale', 'Key', 'BPM',
       'Danceability', 'Positiveness', 'Liveness', 'Energy', 'speechness',
       'KeyScaled', 'Lyrics', 'Not_Formatted_Lyrics', 'Lem_Lyrics',
       'Lyrics_Length', 'Longest_Line', 'Length_Words_Ratio'],
      dtype='object')

In [99]:
df.drop(['Lyrics','Title','Artist','KeyScaled','Not_Formatted_Lyrics'],axis=1,inplace=True)

In [100]:
df.head()

,Genre,Explicit,Length,Scale,Key,BPM,Danceability,Positiveness,Liveness,Energy,speechness,Lem_Lyrics,Lyrics_Length,Longest_Line,Length_Words_Ratio
0,rock,False,5.02,D,Major,106,25,11,13,97,7,awaken rebellion wake change course law follow...,173,47,34.462151
1,rock,False,4.08,F#,Minor,86,32,18,10,28,3,hello darkness old friend I talk vision softly...,218,97,53.431373
2,rock,False,5.00,A,Major,182,38,51,24,95,4,incite riot put trance rotate frame world rely...,154,48,30.800000
3,rock,False,4.15,A,Major,188,31,76,8,70,5,back black I hit sack I long I glad back yes I...,307,55,73.975904
4,rock,False,4.24,G,Minor,170,35,24,12,88,4,mock slaughter pure kind blind noise maze flas...,95,53,22.405660


In [101]:
df["Major"] = df["Key"]
df["Major"] = df["Major"].apply(lambda x: 0 if x == "Minor" else 1)
df.drop(columns=['Key'],inplace=True)

In [102]:
res = pd.get_dummies(df["Scale"])
df.drop(columns=['Scale'],inplace=True)
df = pd.concat([df, res], axis=1)
df.dropna(subset=['Lem_Lyrics'],inplace=True)

In [103]:
#ignore the stopword, ignore words that haven't appeared with 10% of the song
vectorizer=CountVectorizer(stop_words='english',min_df=0.1)
lyrics_features=vectorizer.fit_transform(df.pop("Lem_Lyrics"))

In [109]:
#convertis the sparse matrix to pandas dataframe
sparse=pd.DataFrame.sparse.from_spmatrix(lyrics_features)
df = pd.concat([df, sparse], axis=1)
df.fillna(0)

,Genre,Explicit,Length,BPM,Danceability,Positiveness,Liveness,Energy,speechness,Lyrics_Length,...,61,62,63,64,65,66,67,68,69,70
0,rock,False,5.02,106.0,25.0,11.0,13.0,97.0,7.0,173.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,rock,False,4.08,86.0,32.0,18.0,10.0,28.0,3.0,218.0,...,2.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0
2,rock,False,5.00,182.0,38.0,51.0,24.0,95.0,4.0,154.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,rock,False,4.15,188.0,31.0,76.0,8.0,70.0,5.0,307.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,rock,False,4.24,170.0,35.0,24.0,12.0,88.0,4.0,95.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11394,hiphop,True,3.44,77.0,71.0,90.0,11.0,65.0,25.0,517.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11395,hiphop,False,5.24,75.0,82.0,78.0,4.0,50.0,23.0,603.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11398,hiphop,True,4.14,92.0,75.0,69.0,35.0,79.0,33.0,767.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11401,hiphop,False,3.54,89.0,73.0,58.0,28.0,57.0,45.0,743.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
y=df["Genre"]
X=df.drop(columns=["Genre"],axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)


In [111]:
text_mnb=SVC().fit(X_train,y_train)

d:\idan\python\genre_prediction_data_science\venv\lib\site-packages\sklearn\utils\validation.py:515: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [112]:
ba=text_mnb.predict(X_test)
print(accuracy_score(y_true=y_test,y_pred=ba))

NameError: name 'text_mnb' is not defined